<a href="https://colab.research.google.com/github/jupareto/ENG1111/blob/main/FINAL_RECIarquivo_com_a_rede_1_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Título:** arquivo_com_a_rede

**Versão:** 1.0.1

**Data:** 5/12/2022

**Autores:**
- Carolina Fischberg Blank (2211321)
- Guilherme Fagundes Maximo (2211318)
- Julia Malhães da Cunha Pareto (2211308)
- Pedro de Almeida Barizon (2211350)

**Professor Orientador:** João Ricardo Cortes Nunes

**Monitores:**
- Gabriella Serra Gonçalves
- Vinícius Machado da Rocha Viana

# **Bibliotecas Necessárias**

In [ ]:
# Bibliotecas operacionais:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import drive

# Bibliotecas voltadas à rede neural propriamente dita:

import tensorflow as tf
from tensorflow import keras # A Keras atuará como frontend do TensorFlow, que atuará como backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Verificação do Dataset**

In [ ]:
# Definição do path para o dataset:

drive.mount('/drive')
train_dir = os.path.join('/drive/MyDrive/archive/Garbage classification/Garbage classification')

# Definição da lista de categorias para uma classificação de imagens de forma humanamente compreensível:

labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [ ]:
# Checagem da quantidade de dados disponíveis para o treino da rede neural:

for label in labels:
    directory = os.path.join(train_dir, label)
    print("Images of label \"" + label + "\":\t", len(os.listdir(directory)))

In [ ]:
# Plotagem de imagens de diferentes categorias para entendermos o dataset:

plt.figure(figsize=(30,14))

for i in range(4):
    directory = os.path.join(train_dir, labels[i])
    for j in range(8):
        path = os.path.join(directory, os.listdir(directory)[j])
        img = mpimg.imread(path)
        
        plt.subplot(4, 8, i*8 + j + 1)
        plt.imshow(img)
        
        if j == 0:
            plt.ylabel(labels[i], fontsize=20)
        
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
plt.tight_layout()
plt.show()

In [ ]:
# Checagem do tamanho de uma imagem individualmente:

directory = os.path.join(train_dir, 'cardboard')
path = os.path.join(directory, os.listdir(directory)[0])
image = mpimg.imread(path)
image.shape

# **Definição do Modelo**

In [ ]:
model = tf.keras.Sequential([ # Escolheu-se um modelo de rede sequencial (a informação flui de camada a camada sem retornar)
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(384, 512, 3)), # Camada de aplicação do operador convolucional. Cada um dos 32 neurônios será responsável por uma janela 3x3 vinda da imagem original
  tf.keras.layers.BatchNormalization(), # Normalização dos valores, para que sigam uma distribuição Gaussiana. Trata-se de uma boa prática que permite um melhor funcionamento da rede, facilitando as contas e possibilitando que a rede convirja mais rapidamente
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # A função de ativação ReLU, segundo a literatura especializada, apresenta excelentes resultados (converge rapidamente e é fácil de ser calculada), inclusive em redes neurais convolucionais — daí sua escolha
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2), # Redução da imagem original a partir do escaneamento com uma janela 2x2 que percorre a imagem (a cada posição, dos quatro pixels, apenas o de maior valor comporá a nova imagem)
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # A convulação será aplicada à imagem reduzida, permitindo que padrões mais amplos sejam averiguados de uma só vez
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2), # Nova redução da imagem já reduzida
  tf.keras.layers.Dropout(0.2),
  
  # O dinamismo convolução-redução repete-se nas linhas abaixo...

  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2), # Com mais reduções, os padrões percebidos passam a relacionar mais áreas diferentes da figura original, tornando-se mais complexos — daí o aumento no número de neurônios 32 -> 64 -> 128 ->...
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),

  # ... até a camada de saída

  tf.keras.layers.Flatten(), # As informações anteriores, que se encontravam na forma de tensor (muitas dimensões), são transformadas em um vetor (uma dimensão). Dito de outra forma, o tensor é "achatado" (flattened)
  tf.keras.layers.Dense(512, activation='relu'), # Um último processamento até a camada final
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(6, activation='softmax') # A função de ativação softmax é recomendada para classificações não binárias, porque normaliza os valores de saída, de modo que seu somatório seja 1 (o que permite a interpretação dos resultados como probabilidades)
])

model.summary() # O sumário do modelo mostra seus detalhes camada a camada e, por fim, fornece uma visão panorâmica

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 382, 510, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 382, 510, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 380, 508, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 380, 508, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 190, 254, 32)     0         
 )                                                               
                                                        

# **Compilação do Modelo**


A função de perda (parâmetro "loss") escolhida foi a *sparse_categorical_crossentropy*, uma vez que este modelo trata de um problema de classificação cujas categorias excedem duas possibilidades. Dessa forma, a *binary_crossentropy*, indicada em situações binárias ("A ou B"), seria inadequada.

Além disso, o otimizador escolhido foi o Adam, que, segundo a literatura especializada, apesar de possuir uma generalização inferior à do SGD (método do gradiente estocástico descendente), tende a convergir mais rapidamente. Cumpre ressaltar, aliás, que o parâmetro "lr" corresponde à taxa de aprendizagem (*learning rate*), isto é, ao "tamanho dos passos" com que o Adam percorrerá a hipersuperfície da função de perda em busca de pontos de mínimo. Se for muito elevada, pode ser que o Adam acabe ultrapassando os pontos desejados por estar muito veloz. Se muito baixa, demorará uma eternidade computacional. É importante, pois, o uso de um valor equilibrado.

Por fim, para se verificar o desempenho do modelo, monitora-se sua acurácia, isto é, a razão entre as previsões corretas e as previsões totais feitas com base nos valores do conjunto de treino.

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = 0.0001), metrics=['accuracy'])

NameError: ignored

# **Ampliação do Dataset**

In [ ]:
# Ampliação do dataset por meio de ligeiras modificações nas imagens originais (rotações e inversões, por exemplo), o que aumenta os insights obtíveis pela rede:

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                                   rotation_range=15,zoom_range=0.1,
                                   width_shift_range=0.15,height_shift_range=0.15,
                                   shear_range=0.1,
                                   fill_mode="nearest",
                                   rescale=1./255., 
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(384, 512), batch_size=32, class_mode='binary', subset='training')
validation_generator = train_datagen.flow_from_directory(train_dir, target_size=(384, 512), batch_size=32, class_mode='binary', subset='validation')

# **Preparação para Salvamento dos Pesos Otimizados**

In [ ]:
# Criação de uma função de retorno para que os pesos possam ser salvos:

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# **Treinamento do Modelo**

In [ ]:
history = model.fit(train_generator, epochs=1, verbose=1, validation_data=validation_generator, callbacks=[cp_callback])

In [ ]:
os.listdir(checkpoint_dir)

# **Carregamento dos Pesos Salvos e Avaliação do Modelo**

In [ ]:
# Carregamento dos pesos:
model.load_weights(checkpoint_path)

# Avaliação do modelo:
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# **Salvamento do Modelo e dos Pesos Otimizados**

In [ ]:
# Salvamento dos pesos já otimizados:
model.save_weights('./checkpoints/my_checkpoint')

# Salvamento de todo o modelo como um "SavedModel":
!mkdir -p saved_model
model.save('saved_model/my_model')

# **Recuperação do Modelo Salvo**

In [ ]:
# Recuperação do modelo salvo:
new_model = tf.keras.models.load_model('/drive/MyDrive/saved_model/my_model')

# Checagem da arquitetura do modelo recuperado:
new_model.summary()

# **Previsões do Modelo a partir de Imagens Individuais do Datasest**

In [ ]:
cat = int(input('Enter any category by index: ')) # Escolha de uma categoria a ser testada
ind = int(input('Enter any index to test: ')) # Escolha do índice da imagem a ser testada

directory = os.path.join(train_dir, labels[cat % 6])
try:
    path = os.path.join(directory, os.listdir(directory)[ind])
    img = mpimg.imread(path)

    # Pré-processamento da imagem a ser classificada:

    x = keras.preprocessing.image.img_to_array(img) # Primeiro, é transformada em um tensor (array)
    x = np.expand_dims(x, axis=0) # Por fim, adiciona-se mais um eixo ao tensor, a fim de que seu formato fique compatível com a Keras
    
    images = np.vstack([x])
    classes = new_model.predict(images) # O modelo realiza sua previsão ao retornar um vetor normalizado cujas componentes são as probabilidades (seu somatório equivale a 1) respectivas a cada classe
    pred = labels[np.argmax(classes)] # Com base no índice da maior probabilidade no vetor que contém as previsões, a classe respectiva é escolhida na lista de categorias
    
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.title('Actual: {}      Pred: {}'.format(labels[cat], pred)) # Comparação entre o real e o predito
    
except:
    print('Invalid Value') # Uma mensagem de erro será exibida caso algum dos valores de entrada seja inválido

# **Referências Bibliográficas**

**Nove primeiros vídeos da playlist de redes neurais do canal Universo Discreto, de autoria do cientista da computação Lucas Grassano Lattari:**
- https://youtu.be/WPxV5WXP3OM
- https://youtu.be/NFZwmiKTFfI
- https://youtu.be/tYXGzQs31Og
- https://youtu.be/IqLyqr_eI0U
- https://youtu.be/_SeXVKD0NNk
- https://youtu.be/e5nC31i7nVY
- https://youtu.be/rQ-BePyJK0E
- https://youtu.be/gbrHEsbTdF0
- https://youtu.be/0lvHURoyhtc

**Código do usuário do Kaggle SANKET MATHUR, cujo trabalho serviu-nos de base:**
- https://www.kaggle.com/code/sanketmathur7/garbage-classification-cnn